## 1. Imports

In [1]:
import chromadb
from getpass import getpass
from langchain.storage import create_kv_docstore, LocalFileStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import ParentDocumentRetriever
from langchain.vectorstores import Chroma
import os
import pandas as pd
import requests
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

## 2. Embedding models (local)

In [2]:
# For Apple Silicon users: run the following code to make use of MPS (Apple's Metal Performance Shaders) for faster computation
import torch

# set device to MPS
device = torch.device("mps")

# empty cache and set memory fraction
torch.mps.empty_cache()
torch.mps.set_per_process_memory_fraction(0.9)

# choose embeddings model
multilingual = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

# local embedding model, download to cache folder
embedding_model = SentenceTransformer(multilingual, cache_folder="../Data/sentence_transformers", device=device)

embeddings_retrieve = HuggingFaceEmbeddings(model_name=multilingual, cache_folder="../Data/sentence_transformers")

# move model to MPS
embedding_model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [ ]:
# if not using apple silicon, use this code

# # local embedding model, download to cache folder
# multilingual = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
# 
# # model used for document embedding
# embedding_model = SentenceTransformer(
#     model_name_or_path=multilingual, 
#     cache_folder="../Data/sentence_transformers"
# )

In [4]:
# # model used for query embedding
# embeddings_retrieve = HuggingFaceEmbeddings(
#     model_name=multilingual,
#     cache_folder="../Data/sentence_transformers"
# )

## 3. Chroma client setup

In [5]:
# initiate the chroma client, which is the interface to the database
database_path = "../Data/my_vectordb"
chroma_client = chromadb.PersistentClient(path=database_path)

In [6]:
# print the collections
chroma_client.list_collections()

[Collection(name=tenderned),
 Collection(name=binnenlands_bestuur),
 Collection(name=rijksoverheid),
 Collection(name=ibestuur)]

## 4. Retrievers for all databases

4.1 Parent / child splitters

In [ ]:
# define the retrievers, parent and child splitters, also change files_to_database.py
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                                 chunk_overlap=0)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=128,
                                                chunk_overlap=0)

4.2 Rijksoverheid retriever

In [8]:
# rijksoverheid database
rijksoverheid_db= Chroma(
    collection_name="rijksoverheid",
    client=chroma_client,
    persist_directory="../Data/my_vectordb",
    embedding_function=embeddings_retrieve,
)

In [9]:
# rijksoverheid retriever
full_path = os.path.abspath("../Data/my_vectordb/full_documents/rijksoverheid")
fs = LocalFileStore(full_path)
store = create_kv_docstore(fs)

rijksoverheid_db_retriever = ParentDocumentRetriever(
    vectorstore=rijksoverheid_db,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    enable_limit=True
    )

4.3 ibestuur retriever

In [10]:
# ibestuur database
ibestuur_db= Chroma(
    collection_name="ibestuur",
    client=chroma_client,
    persist_directory="../Data/my_vectordb",
    embedding_function=embeddings_retrieve,
)

In [ ]:
# ibestuur retriever
full_path = os.path.abspath("../Data/my_vectordb/full_documents/ibestuur")
fs = LocalFileStore(full_path)
store = create_kv_docstore(fs)

ibestuur_db_retriever = ParentDocumentRetriever(
    vectorstore=ibestuur_db,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    )

4.4 binnenlandsbestuur retriever

In [12]:
# binnenlandsbestuur database
binnenlandsbestuur_db= Chroma(
    collection_name="binnenlands_bestuur",
    client=chroma_client,
    persist_directory="../Data/my_vectordb",
    embedding_function=embeddings_retrieve,
)

In [ ]:
# binnenlandsbestuur retriever
full_path = os.path.abspath("../Data/my_vectordb/full_documents/binnenlands_bestuur")
fs = LocalFileStore(full_path)
store = create_kv_docstore(fs)

binnenlandsbestuur_db_retriever = ParentDocumentRetriever(
    vectorstore=binnenlandsbestuur_db,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    )

4.5 tenderned retriever

In [14]:
# tenderned database
tenderned_db= Chroma(
    collection_name="tenderned",
    client=chroma_client,
    persist_directory="../Data/my_vectordb",
    embedding_function=embeddings_retrieve,
)

In [ ]:
# tenderned retriever
full_path = os.path.abspath("../Data/my_vectordb/full_documents/tenderned")
fs = LocalFileStore(full_path)
store = create_kv_docstore(fs)

tenderned_db_retriever = ParentDocumentRetriever(
    vectorstore=tenderned_db,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    )

## 5. Set up inference pipeline

In [76]:
# Model GEITJE-7B-ULTRA (PRIVATE SERVER / COSTS $$$ TO RUN ... )
API_URL = "https://oi6h8u843v8nt5qt.eu-west-1.aws.endpoints.huggingface.cloud"
API_KEY = getpass("Enter your API KEY:")

In [17]:
# # Model ZEPHYR-7B-ALPHA (PUBLIC SERVER / FREE TO RUN ... )
# API_URL = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-alpha"
# API_KEY = getpass("Enter your Hugging Face API Token:")

In [18]:
headers = {
	"Accept" : "application/json",
	"Authorization": "Bearer " + API_KEY,
    "Content-Type": "application/json" 
}

In [19]:
def query(prompt, parameters=None):
	if parameters is None:
		parameters = {
			"max_new_tokens": 128,
			"temperature": 0.5,
			"top_p": 0.99,
			"top_k": 50,
			"repetition_penalty": 1.,
			"wait_for_model": True,
			"do_sample": False
		}
			
	payload = {
		"inputs": prompt,
		"parameters": parameters
	}
	
	response = requests.post(API_URL, headers=headers, json=payload)
	# response successful?
	response.raise_for_status()
	return response.json()

## 6. Load Framework

In [91]:
# prevent reading extra unnamed column
framework = pd.read_csv("../Results/framework_questions_translated.csv", usecols=[' #', '2022 GTMI Indicators & Sub-indicators NL', 'Response options & Data format NL'])


In [21]:
framework.head()

,#,2022 GTMI Indicators & Sub-indicators NL,Response options & Data format NL
0,I-1,Is er een gedeeld cloud platform beschikbaar v...,"0= Nee, 1= Alleen cloud strategie/beleid (nog ..."
1,I-1.1,Naam van het Overheids Cloud platform,Tekst
2,I-1.2,Cloud platform / strategie URL,URL
3,I-1.3,Overheids Cloud gelanceerd / zal worden gelanc...,YYYY
4,I-1.4,Type beschikbaar cloud platform,"0= Onbekend, 1= Publiek (Commercieel), 2= Priv..."


## 7. Define functions to operationalize framework

7.1 Prompt template

In [99]:
# # chat prompt template without prompt engineering
template = """
Antwoord volgens het data-format met behulp van de context.  \n\n"

CONTEXT: {context}

DATA FORMAT: {data_format}

VRAAG: {question}

ANTWOORD: 
"""

In [100]:
# # # chat prompt template with prompt engineering
# template = """
# Je bent 'GovTech-GPT', een geavanceerde AI-assistent met uitgebreide expertise in digitale technologieën specifiek gericht op toepassingen binnen de Nederlandse overheid. Je belangrijkste taak is het ondersteunen bij het operationaliseren van e-gov benchmarking frameworks. Je antwoordt altijd op basis van de meest recente gegevens en inzichten, en houdt rekening met de specifieke context van de Nederlandse overheid. Antwoorden geef je alleen volgens het gespecificeerde dataformat, waarbij je, indien mogelijk, het cijfer gebruikt en niet de tekst. Voeg verder geen enkele tekst, toelichting of uitleg meer toe. Als je het antwoord niet weet, geef je geen fictieve informatie of uitleg, maar antwoord enkel en alleen met: 'Geen antwoord.'  \n\n"
# 
# CONTEXT: {context}
# 
# DATA FORMAT: {data_format}
# 
# VRAAG: {question}
# 
# ANTWOORD: 
# """

7.2 Process context

In [78]:
def process_context(context):
    content = ""
    meta_data = []
    for doc in context:
        content += str(doc.page_content) + "\n"
        meta_data.append(doc.metadata)
    
    return content, meta_data

In [79]:
def format_context(rijksoverheid_content, ibestuur_content, binnenlandsbestuur_content, tenderned_content):
    # return the content as a string seperated by a new line and a title for each source
    return (f"Rijksoverheid:\n{rijksoverheid_content}\n"
            f"iBestuur:\n{ibestuur_content}\n"
            f"Binnenlands Bestuur:\n{binnenlandsbestuur_content}\n"
            f"Tenderned:\n{tenderned_content}")

In [80]:
rijksoverheid_db_retriever.search_kwargs = {"k": 1}
ibestuur_db_retriever.search_kwargs = {"k": 1}
binnenlandsbestuur_db_retriever.search_kwargs = {"k": 1}
tenderned_db_retriever.search_kwargs = {"k": 1}

7.3 Retrieve context

In [81]:
def retrieve_context(question):
    rijksoverheid_context = rijksoverheid_db_retriever.invoke(question)
    ibestuur_context = ibestuur_db_retriever.invoke(question)
    binnenlandsbestuur_context = binnenlandsbestuur_db_retriever.invoke(question)
    tenderned_context = tenderned_db_retriever.invoke(question)
    
    rijksoverheid_content, rijksoverheid_meta_data = process_context(rijksoverheid_context)
    ibestuur_content, ibestuur_meta_data = process_context(ibestuur_context)
    binnenlandsbestuur_content, binnenlandsbestuur_meta_data = process_context(binnenlandsbestuur_context)
    tenderned_content, tenderned_meta_data = process_context(tenderned_context)
    
    return format_context(rijksoverheid_content, ibestuur_content, binnenlandsbestuur_content, tenderned_content)

## 8. Operationalize framework

In [82]:
def get_main_indicator(index):
    if '.' in index:
        return index.split('.')[0]
    return index

In [83]:
def operationalize_framework(framework, template):
    framework_operationalized = framework.copy()
    framework_operationalized["Operationalisatie"] = None
    framework_operationalized['Prompt'] = None
    framework_operationalized[' #'] = framework_operationalized[' #'].str.replace('I-', '')

    for index, row in tqdm(framework_operationalized.iterrows(), total=framework_operationalized.shape[0]):
        idx = row[' #']
        main_indicator_idx = get_main_indicator(idx)
        sub_indicator_idx = idx.split('.')[1] if '.' in idx else None
        indicator_info = row['2022 GTMI Indicators & Sub-indicators NL']
        data_format = row['Response options & Data format NL']
        
        if sub_indicator_idx:
            # look up the main indicator
            main_indicator = framework_operationalized.loc[framework_operationalized[' #'] == main_indicator_idx]['2022 GTMI Indicators & Sub-indicators NL'].iloc[0]
            full_question = f"{main_indicator}, indien ja, {indicator_info}?"
            prompt = template.format(
                context=retrieve_context(full_question), 
                data_format=data_format, 
                question=full_question  # main indicator + sub indicator
            )
        else:
            prompt = template.format(
                context=retrieve_context(indicator_info), 
                data_format=data_format, 
                question=indicator_info
            )
        
        # if http error, output = HTTP error
        try:
            output = query(prompt)
        except requests.exceptions.HTTPError as e:
            error_message = f"HTTP error: {e.response.status_code}"
            output = [{'generated_text': error_message}]
        
        framework_operationalized.loc[framework_operationalized[' #'] == idx, 'Operationalisatie'] = output[0]['generated_text']
        framework_operationalized.loc[framework_operationalized[' #'] == idx, 'Prompt'] = prompt
    
    return framework_operationalized

## 9. Run

In [101]:
df = operationalize_framework(framework, template)

100%|██████████| 9/9 [00:46<00:00,  5.14s/it]


In [103]:
df

,#,2022 GTMI Indicators & Sub-indicators NL,Response options & Data format NL,Operationalisatie,Prompt
141,14,Is er een online systeem voor het beheer van P...,"0= Nee, 1= Implementatie in uitvoering, 2= Ja ...",1 (in gebruik)\n\nDeze contexten en vragen zij...,\nAntwoord volgens het data-format met behulp ...
142,14.1,Naam van het online systeem voor het beheer v...,Tekst,"\nJa, er is een online systeem voor het beheer...",\nAntwoord volgens het data-format met behulp ...
143,14.2,URL van het CMIS portaal,URL,"Ja, er is een online systeem voor het beheer v...",\nAntwoord volgens het data-format met behulp ...
144,14.3,online systeem voor het beheer van Publieke i...,YYYY,"\nJa, er is een online systeem voor het beheer...",\nAntwoord volgens het data-format met behulp ...
145,14.4,Type software van online systeem voor het beh...,"0= Onbekend, 1= Maatwerk Software, 2= Commerci...",\nOp basis van de gegeven context en documente...,\nAntwoord volgens het data-format met behulp ...
146,14.5,functionele mogelijkheden van online systeem ...,"0= Onbekend, 1= Alleen Publieke Investeringen ...",Antwoord volgens het data-format met behulp va...,\nAntwoord volgens het data-format met behulp ...
147,14.6,Wisselt online systeem voor het beheer van Pu...,"0= Nee, 1= Ja (via verschillende interfaces) 2...",\nOp basis van de gegeven context en de beschr...,\nAntwoord volgens het data-format met behulp ...
148,14.7,Publicatie van gegevens online systeem voor he...,"0= Nee, 1= Ja (intern, niet gepubliceerd), 2=...","\n""\n\n1= Ja (publiek, gepubliceerd)\n\nDe ove...",\nAntwoord volgens het data-format met behulp ...
149,14.7.1,Zo ja > Ondersteunend document (rapport / URL),Voer URL in (publieke link) of Voeg relevant r...,<Voorbeeld van een antwoord afhankelijk van de...,\nAntwoord volgens het data-format met behulp ...


In [36]:
# To look through the results
for index, row in df.iterrows():
	print(f"PROMPT: {row['Prompt']}")
	print(f"{row['Operationalisatie']}")
	print("\n")

PROMPT: 
Antwoord volgens het data-format met behulp van de context.  

"

CONTEXT: Rijksoverheid:
.    Hybrid & Multi-cloud    Nu de migratie van grote ondernemingen naar de cloud goed op gang is gekomen, kunnen  we zien dat de nadruk op kosten en prestaties geleidelijk is verschoven naar andere kritieke  vereisten, zoals interoperabiliteit, dataportabiliteit, anti-vendor lock-in en beveiliging. Om deze  doelen te bereiken, lijkt de cloud-industrie zich erop te richten bedrijven in staat te stellen hun  werklast te verdelen over meerdere computeromgevingen, of het nu gaat om privé- en openbare  clouds (hybrid cloud) of om meerdere openbare clouds (multi-cloud).    Een belangrijke deeloplossing bij het creëren van deze infra-agnostische applicaties is het  gebruik van containers. Containers zorgen voor de snelle en eenvoudige implementatie van  discrete applicatiecomponenten die in praktische iedere cloudomgeving kunnen worden  uitgevoerd

iBestuur:
Als overheid naar de publieke cloud,

# 10.  Save data to CSV file

In [42]:
df.to_csv("../Results/operationalization-PCF.csv", index=False)